# Auto Builder
AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we introduce a new class, `AgentCreator`, to help user build an automatic task solving process powered by multi-agent system. Specifically, our building pipeline include `init()`, `build()`, and `start()`. In `build()`, we prompt a gpt-4 model to create multiple participant agent and initialize a group chat, and specify whether this task need programming to solve. After that, user can call `start()` at a proper time to complete the task. 

## Requirement

Auto Agent setup need `vllm>=0.2.0` and `fastchat>=0.2.30`, you can use the following command to install the latest version of [vLLM](https://github.com/vllm-project/vllm) and [FastChat](https://github.com/lm-sys/FastChat).
```
pip install vllm "fastchat[model_worker]"
```

In [ ]:
!pip install vllm "fastchat[model_worker]"

## Specify a task with detailed information
Describe your task in natural language and specify your target (e.g., find some paper from arxiv).

In [1]:
task="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."

## Prepare configuration
Prepare a `config_path` for assistant agent to limit the choice of LLM you want to use in this task. This config can be a path of json file or a name of environment variable. A `default_llm_config` is also required for initialize the specific config of LLMs like seed, temperature, etc...

In [2]:
config_path = '/home/elpis_ubuntu/LLM/autogen/OAI_CONFIG_LIST'
default_llm_config = {
    'temperature': 0
}

## Complete the task building
Then, we can use the `AgentCreator` to automatically complete the above task. First, initialize the AgentCreator with the predefined `task` and `config_path`. Then, call the `build` with the `default_llm_config`. GPT-4 will automatically create multiple agents and initialize a groupchat that use to complete this task. Finally, call the `start()` to start the task completion process with a groupchat. When the task begin, groupchat content between each agent will be shown in the console. This process will become interactive when the groupchat manager think that is necessary. Note that if this process involve some open-sourced LLMs (for example, LLaMA 2), an endpoint server will be automatically set up with a background process. You can call the `clear_all()` to wipe them out after the task completed.

In [3]:
from autobuild import AgentCreator


administrator = AgentCreator(
    task,
    config_path=config_path
)
administrator.build(default_llm_config)
administrator.start()
administrator.clear_all()


Initializing usable port...
57505 ports found.


ModuleNotFoundError: No module named 'openai.openai_object'